# Analysis of A05 results

## Data loading

In [1]:
from curve_analyzer.utils.data_processing import load_curves, load_trait, filter_df
curves = load_curves() # load all curves
df_a05 = load_trait("a05") # load all results for trait a05
df_a05 = df_a05.merge(curves, "left", "curve") # join curve metadata to trait results

In [7]:
df = filter_df(df_a05, bitlength=256, cofactor=1)
df = df[df["l"] == 3] # filter out only results for parameter l == 3

## Data inspection

In [8]:
df

,curve,l,least,full,relative,cofactor,bitlength,simulated
8301,secp256r1,3,2.0,6.0,3.0,1,256.0,False
9113,x962_sim_256_seed_diff_1112118,3,2.0,6.0,3.0,1,256.0,True
9117,x962_sim_256_seed_diff_1273169,3,2.0,2.0,1.0,1,256.0,True
9121,x962_sim_256_seed_diff_1316428,3,4.0,4.0,1.0,1,256.0,True
9125,x962_sim_256_seed_diff_1320584,3,2.0,6.0,3.0,1,256.0,True
...,...,...,...,...,...,...,...,...
877269,brainpoolP256r1,3,8.0,8.0,1.0,1,256.0,False
877273,FRP256v1,3,8.0,8.0,1.0,1,256.0,False
877277,256-bit Random ECP Group,3,2.0,6.0,3.0,1,256.0,False
877281,Fp256BN,3,2.0,6.0,3.0,1,256.0,False


In [ ]:
df.agg(set) # display only distinct values

In [ ]:
df.describe()

## Data visualization

In [ ]:
from curve_analyzer.utils.visualization import normalized_barplot, normalized_bubbleplot

In [ ]:
for column in ("least", "full"):
    normalized_barplot(df, column, title=f"Normalized barplot of {column}")

In [ ]:
for l in (2, 3):
    normalized_bubbleplot(df[df["l"] == l], f"least", f"full", title=f"Normalized bubble plot for l={l}")

## Playground